In [11]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
import tensorflow.keras.backend as K
import numpy as np
from tensorflow import keras
import time
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import RandomFourierFeatures
import pandas as pd
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from __future__ import division
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [15]:
dfTrainz = pd.read_csv('SVMData/train_feat.csv', index_col=0)
dfTestz = pd.read_csv('SVMData/test_feat.csv', index_col=0)
dfTrainz.shape, dfTestz.shape
dfTrainz.dtypes
dataset = pd.concat([dfTrainz,dfTestz])
dataset.shape, dfTrainz.shape, dfTestz.shape
dataset['gender'].unique()
dataset

,all#count,session#count,seek_video#num,play_video#num,pause_video#num,stop_video#num,load_video#num,problem_get#num,problem_check#num,problem_save#num,...,truth,username,course_id,age,gender,education,user_enroll_num,course_enroll_num,cluster_label,course_category
enroll_id,,,,,,,,,,,,,,,,,,,,,
772,-0.228387,-0.228387,-0.243043,-0.296739,-0.333535,-0.059298,-0.480968,-0.042412,-0.305844,-0.202161,...,1,5981,course-v1:TsinghuaX+70800232X+2015_T2,1.504677,1,3,-0.007581,-0.138544,2,0
773,-0.204883,-0.204883,-0.115253,-0.242027,-0.296012,-0.059298,-0.445267,-0.042412,-0.305844,-0.202161,...,1,1544995,course-v1:TsinghuaX+70800232X+2015_T2,-0.599969,0,0,-0.360012,-0.138544,2,0
774,-0.139320,-0.139320,-0.051358,-0.105248,-0.208460,-0.057709,-0.195358,-0.042412,-0.305844,-0.202161,...,1,1072798,course-v1:TsinghuaX+70800232X+2015_T2,-0.599969,0,0,-0.095689,-0.138544,2,0
776,-0.207357,-0.207357,-0.243043,-0.255705,-0.296012,-0.057709,-0.409566,-0.042412,-0.305844,-0.202161,...,0,561867,course-v1:TsinghuaX+70800232X+2015_T2,2.085269,2,6,-0.007581,-0.138544,2,0
777,-0.225912,-0.225912,-0.243043,-0.296739,-0.333535,-0.059298,-0.480968,-0.042412,-0.305844,-0.202161,...,1,1368125,course-v1:TsinghuaX+70800232X+2015_T2,-0.599969,0,0,0.432957,-0.138544,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393203,-0.225912,-0.225912,-0.243043,-0.296739,-0.321027,-0.059298,-0.445267,-0.042412,-0.305844,-0.202161,...,1,29394,course-v1:UC_BerkeleyX+ColWri2_1x_2015_T1+2016_TS,1.432103,2,3,-0.007581,-0.695476,2,0
393204,-0.225912,-0.225912,-0.243043,-0.296739,-0.321027,-0.059298,-0.445267,-0.042412,-0.305844,-0.202161,...,1,3238122,course-v1:UC_BerkeleyX+ColWri2_1x_2015_T1+2016_TS,-0.599969,0,0,-0.183797,-0.695476,2,0
131061,-0.228387,-0.228387,-0.243043,-0.296739,-0.333535,-0.059298,-0.480968,-0.042412,-0.305844,-0.202161,...,1,1103805,course-v1:TsinghuaX+20220053X_2015_T2+2015_T2,-0.599969,0,0,-0.271904,-0.740180,2,0


In [16]:
dataset = pd.get_dummies(dataset, columns=['gender'])
dataset.head()

,all#count,session#count,seek_video#num,play_video#num,pause_video#num,stop_video#num,load_video#num,problem_get#num,problem_check#num,problem_save#num,...,course_id,age,education,user_enroll_num,course_enroll_num,cluster_label,course_category,gender_0,gender_1,gender_2
enroll_id,,,,,,,,,,,,,,,,,,,,,
772,-0.228387,-0.228387,-0.243043,-0.296739,-0.333535,-0.059298,-0.480968,-0.042412,-0.305844,-0.202161,...,course-v1:TsinghuaX+70800232X+2015_T2,1.504677,3,-0.007581,-0.138544,2,0,0,1,0
773,-0.204883,-0.204883,-0.115253,-0.242027,-0.296012,-0.059298,-0.445267,-0.042412,-0.305844,-0.202161,...,course-v1:TsinghuaX+70800232X+2015_T2,-0.599969,0,-0.360012,-0.138544,2,0,1,0,0
774,-0.139320,-0.139320,-0.051358,-0.105248,-0.208460,-0.057709,-0.195358,-0.042412,-0.305844,-0.202161,...,course-v1:TsinghuaX+70800232X+2015_T2,-0.599969,0,-0.095689,-0.138544,2,0,1,0,0
776,-0.207357,-0.207357,-0.243043,-0.255705,-0.296012,-0.057709,-0.409566,-0.042412,-0.305844,-0.202161,...,course-v1:TsinghuaX+70800232X+2015_T2,2.085269,6,-0.007581,-0.138544,2,0,0,0,1
777,-0.225912,-0.225912,-0.243043,-0.296739,-0.333535,-0.059298,-0.480968,-0.042412,-0.305844,-0.202161,...,course-v1:TsinghuaX+70800232X+2015_T2,-0.599969,0,0.432957,-0.138544,3,0,1,0,0


In [17]:
dataset = pd.get_dummies(dataset, columns=['education'])
dataset.head()

,all#count,session#count,seek_video#num,play_video#num,pause_video#num,stop_video#num,load_video#num,problem_get#num,problem_check#num,problem_save#num,...,gender_1,gender_2,education_0,education_1,education_2,education_3,education_4,education_5,education_6,education_7
enroll_id,,,,,,,,,,,,,,,,,,,,,
772,-0.228387,-0.228387,-0.243043,-0.296739,-0.333535,-0.059298,-0.480968,-0.042412,-0.305844,-0.202161,...,1,0,0,0,0,1,0,0,0,0
773,-0.204883,-0.204883,-0.115253,-0.242027,-0.296012,-0.059298,-0.445267,-0.042412,-0.305844,-0.202161,...,0,0,1,0,0,0,0,0,0,0
774,-0.139320,-0.139320,-0.051358,-0.105248,-0.208460,-0.057709,-0.195358,-0.042412,-0.305844,-0.202161,...,0,0,1,0,0,0,0,0,0,0
776,-0.207357,-0.207357,-0.243043,-0.255705,-0.296012,-0.057709,-0.409566,-0.042412,-0.305844,-0.202161,...,0,1,0,0,0,0,0,0,1,0
777,-0.225912,-0.225912,-0.243043,-0.296739,-0.333535,-0.059298,-0.480968,-0.042412,-0.305844,-0.202161,...,0,0,1,0,0,0,0,0,0,0


In [18]:
target = dataset.pop('truth')
trash= dataset.pop('course_id')
trash2= dataset.pop('course_category')
trash3 = dataset.pop('cluster_label')
dataset.head()


,all#count,session#count,seek_video#num,play_video#num,pause_video#num,stop_video#num,load_video#num,problem_get#num,problem_check#num,problem_save#num,...,gender_1,gender_2,education_0,education_1,education_2,education_3,education_4,education_5,education_6,education_7
enroll_id,,,,,,,,,,,,,,,,,,,,,
772,-0.228387,-0.228387,-0.243043,-0.296739,-0.333535,-0.059298,-0.480968,-0.042412,-0.305844,-0.202161,...,1,0,0,0,0,1,0,0,0,0
773,-0.204883,-0.204883,-0.115253,-0.242027,-0.296012,-0.059298,-0.445267,-0.042412,-0.305844,-0.202161,...,0,0,1,0,0,0,0,0,0,0
774,-0.139320,-0.139320,-0.051358,-0.105248,-0.208460,-0.057709,-0.195358,-0.042412,-0.305844,-0.202161,...,0,0,1,0,0,0,0,0,0,0
776,-0.207357,-0.207357,-0.243043,-0.255705,-0.296012,-0.057709,-0.409566,-0.042412,-0.305844,-0.202161,...,0,1,0,0,0,0,0,0,1,0
777,-0.225912,-0.225912,-0.243043,-0.296739,-0.333535,-0.059298,-0.480968,-0.042412,-0.305844,-0.202161,...,0,0,1,0,0,0,0,0,0,0


In [19]:

# Spliting into 80% for training set and 20% for testing set so we can see our accuracy
X_train, x_test, Y_train, y_test = train_test_split(dataset, target, test_size=0.2, random_state=0)


In [ ]:
from sklearn.svm import SVR
regression_model = SVR(kernel='linear', C=1.0)
regression_model.fit(X_train, Y_train)

In [5]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''

    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()

    # Print the results
    print("Time elapsed {} seconds".format(time.time() - start_time))


def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''

    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()

    # Print and return results
    print("Time elapsed {} seconds".format(time.time() - start_time))
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''

    # Indicate the classifier and the training set size
   # print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))

    # Train the classifier
    train_classifier(clf, X_train, y_train)

    # Print the results of prediction for both training and testing
    print ("F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train)))
    print ("F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test)))

In [6]:

# Initialize the three models
clf_A = SVC()
clf_B = KNeighborsClassifier(n_neighbors=3)
clf_C = AdaBoostClassifier(n_estimators=10)

In [9]:
 clf_A.fit(X_train, y_train, batch_size=32, validation_split=0.2)

TypeError: fit() got an unexpected keyword argument 'batch_size'